In [5]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
print('See X:')
print(X)
print('See Y:')
print(y)

See X:
[[ 0.03807591  0.05068012  0.06169621 ... -0.00259226  0.01990842
  -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 ... -0.03949338 -0.06832974
  -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 ... -0.00259226  0.00286377
  -0.02593034]
 ...
 [ 0.04170844  0.05068012 -0.01590626 ... -0.01107952 -0.04687948
   0.01549073]
 [-0.04547248 -0.04464164  0.03906215 ...  0.02655962  0.04452837
  -0.02593034]
 [-0.04547248 -0.04464164 -0.0730303  ... -0.03949338 -0.00421986
   0.00306441]]
See Y:
[151.  75. 141. 206. 135.  97. 138.  63. 110. 310. 101.  69. 179. 185.
 118. 171. 166. 144.  97. 168.  68.  49.  68. 245. 184. 202. 137.  85.
 131. 283. 129.  59. 341.  87.  65. 102. 265. 276. 252.  90. 100.  55.
  61.  92. 259.  53. 190. 142.  75. 142. 155. 225.  59. 104. 182. 128.
  52.  37. 170. 170.  61. 144.  52. 128.  71. 163. 150.  97. 160. 178.
  48. 270. 202. 111.  85.  42. 170. 200. 252. 113. 143.  51.  52. 210.
  65. 141.  55. 134.  42. 111.  98. 164.  48.  96.  90. 162. 150. 27

In [5]:
import azureml.core
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails

import pandas as pd
import numpy as np

print("SDK version:", azureml.core.VERSION)

SDK version: 1.40.0


In [6]:
from azureml.core.authentication import InteractiveLoginAuthentication

# Get the Workspace object from Azure
# You can find tenant id under azure active directory->properties
tenant_id = '198c7d8c-e010-45ce-a018-ec2d9a33f58f'
ia = InteractiveLoginAuthentication(tenant_id=tenant_id)
ws_name = 'automlbook'
subscription_id = '4d278f3d-b4fd-4fa2-86b6-d34b96bc888f'
resource_group = 'Foxy_Resources'
ws = Workspace.get(name=ws_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,
                   auth=ia)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/198c7d8c-e010-45ce-a018-ec2d9a33f58f/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
automlbook
Foxy_Resources
centralus
4d278f3d-b4fd-4fa2-86b6-d34b96bc888f


In [8]:
# Create datastore, try getting datastore via Workspace object
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [9]:
# Create a dataset from the datastore of the Workspace
dataset_name = 'automlbook Titanic Training Data A'
# dataset = Dataset.get_by_name(ws, dataset_name)
dataset = Dataset.get_by_name(ws, dataset_name, version = 'latest')

# Show a sample of the data in the dataset
dataset.take(10).to_pandas_dataframe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,False,3,male,22.0,1,0,7.2500,None,S
1,True,1,female,38.0,1,0,71.2833,C85,C
2,True,3,female,26.0,0,0,7.9250,None,S
3,True,1,female,35.0,1,0,53.1000,C123,S
4,False,3,male,35.0,0,0,8.0500,None,S
5,False,3,male,NaN,0,0,8.4583,None,Q
6,False,1,male,54.0,0,0,51.8625,E46,S
7,False,3,male,2.0,3,1,21.0750,None,S
8,True,3,female,27.0,0,2,11.1333,None,S
9,True,2,female,14.0,1,0,30.0708,None,C


In [10]:
# Set (User Managed) Local Environment up
from azureml.core import Environment

# Editing a run configuration property on-fly.
user_managed_env = Environment("user-managed-env")

user_managed_env.python.user_managed_dependencies = True

# You can choose a specific Python environment by pointing to a Python path 
#user_managed_env.python.interpreter_path = '/home/johndoe/miniconda3/envs/myenv/bin/python'

In [ ]:
# Run Training Experiment locally
from azureml.core import ScriptRunConfig

# Experiment
experiment_name = 'Local_Training_AutoML'
experiment = Experiment(workspace=ws, name=experiment_name)

# Define Compute Cluster to use
compute_target = 'local'
script_path = 'scripts/localTrainingAutoML.py'
dataset_name = 'automlbook Titanic Training Data A'

scriptRunConfig = ScriptRunConfig(
        source_directory='./', 
        command=['python', script_path, dataset_name],
        environment=user_managed_env,
        compute_target=compute_target)
        
AutoML_run = experiment.submit(scriptRunConfig, show_output = True)
RunDetails(AutoML_run).show()

In [ ]:
# Register Model from the AutoML_run
# description = "Best Local AutoML Regression Run using Titanic Sample Data."
# tags = {
#   "project" : "Local Training AutoML", 
#   "creator": "fox", 
#   "task": "classification", 
#   "dataset": "automlbook Titanic Training Data A", 
#   "metric": "normalized_root_mean_squared_error"
# }
# model_name = "Local-Training-AutoML-Titanic-RMSE"

# AutoML_run.register_model(model_name=model_name, description=description, tags=tags)